# Proyecto: Predicción de Churn en Beta Bank

## Descripción del Proyecto
Los clientes de Beta Bank se están yendo, cada mes, poco a poco. Los banqueros descubrieron que es más barato salvar a los clientes existentes que atraer nuevos.

**Objetivo:** Predecir si un cliente dejará el banco pronto. Para ello, utilizaremos un modelo con el máximo valor F1 posible (al menos 0.59). Además, mediremos la métrica AUC-ROC y la compararemos con el valor F1.

## 1. Preparación de los Datos

### 1.1 Importación de Librerías

In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.impute import SimpleImputer

### 1.2 Carga de Datos

In [23]:
# Cargar el dataset
data = pd.read_csv('Churn.csv')

# Mostrar las primeras filas del dataset
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


### 1.3 Análisis Inicial de los Datos

In [24]:
# Información general del dataset
data.info()

# Estadísticas descriptivas
data.describe()

# Verificar valores nulos
data.isnull().sum()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


RowNumber            0
CustomerId           0
Surname              0
CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64

### 1.4 Preprocesamiento de Datos

In [25]:
# Eliminar columnas irrelevantes
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

# Codificación de variables categóricas
data = pd.get_dummies(data, columns=['Geography', 'Gender'], drop_first=True)

# Separar características y objetivo
X = data.drop('Exited', axis=1)
y = data['Exited']

# Dividir el dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

## 2. Análisis del Desequilibrio de Clases

### 2.1 Distribución de Clases

In [26]:
# Distribución de la variable objetivo
y_train.value_counts(normalize=True)

# Imputar valores NaN con la media de cada columna
imputer = SimpleImputer(strategy='mean')
X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)

**Observación:** El conjunto de datos está desequilibrado, con aproximadamente un 80% de clientes que no se han ido y un 20% que sí.

### 2.2 Entrenamiento del Modelo sin Corregir el Desequilibrio

In [27]:
# Crear un pipeline para el modelo de regresión logística
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(random_state=42))
])

# Entrenar el modelo
pipeline.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred = pipeline.predict(X_test)

# Evaluar el modelo
print("F1 Score:", f1_score(y_test, y_pred))
print("AUC-ROC:", roc_auc_score(y_test, pipeline.predict_proba(X_test)[:, 1]))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

F1 Score: 0.2873134328358209
AUC-ROC: 0.7749089613496394
[[1541   52]
 [ 330   77]]
              precision    recall  f1-score   support

           0       0.82      0.97      0.89      1593
           1       0.60      0.19      0.29       407

    accuracy                           0.81      2000
   macro avg       0.71      0.58      0.59      2000
weighted avg       0.78      0.81      0.77      2000



**Observación:** El modelo tiene un F1 score bajo debido al desequilibrio de clases.

## 3. Mejora del Modelo: Corrección del Desequilibrio de Clases

### 3.1 Técnica 1: Sobremuestreo con SMOTE

In [28]:
# Pipeline con SMOTE
smote_pipeline = ImbPipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('classifier', LogisticRegression(random_state=42))
])

# Entrenar el modelo
smote_pipeline.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred_smote = smote_pipeline.predict(X_test)

# Evaluar el modelo
print("F1 Score con SMOTE:", f1_score(y_test, y_pred_smote))
print("AUC-ROC con SMOTE:", roc_auc_score(y_test, smote_pipeline.predict_proba(X_test)[:, 1]))
print(confusion_matrix(y_test, y_pred_smote))
print(classification_report(y_test, y_pred_smote))

F1 Score con SMOTE: 0.5030782761653474
AUC-ROC con SMOTE: 0.7772857603366079
[[1149  444]
 [ 121  286]]
              precision    recall  f1-score   support

           0       0.90      0.72      0.80      1593
           1       0.39      0.70      0.50       407

    accuracy                           0.72      2000
   macro avg       0.65      0.71      0.65      2000
weighted avg       0.80      0.72      0.74      2000



### 3.2 Técnica 2: Submuestreo con RandomUnderSampler

In [29]:
# Pipeline con RandomUnderSampler
under_pipeline = ImbPipeline([
    ('scaler', StandardScaler()),
    ('under', RandomUnderSampler(random_state=42)),
    ('classifier', LogisticRegression(random_state=42))
])

# Entrenar el modelo
under_pipeline.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred_under = under_pipeline.predict(X_test)

# Evaluar el modelo
print("F1 Score con Submuestreo:", f1_score(y_test, y_pred_under))
print("AUC-ROC con Submuestreo:", roc_auc_score(y_test, under_pipeline.predict_proba(X_test)[:, 1]))
print(confusion_matrix(y_test, y_pred_under))
print(classification_report(y_test, y_pred_under))

F1 Score con Submuestreo: 0.49531914893617024
AUC-ROC con Submuestreo: 0.7746961136791645
[[1116  477]
 [ 116  291]]
              precision    recall  f1-score   support

           0       0.91      0.70      0.79      1593
           1       0.38      0.71      0.50       407

    accuracy                           0.70      2000
   macro avg       0.64      0.71      0.64      2000
weighted avg       0.80      0.70      0.73      2000



**Observación:** Ambas técnicas mejoran el F1 score, pero SMOTE parece ser más efectivo.

## 4. Prueba Final con el Mejor Modelo

### 4.1 Entrenamiento del Modelo Final

In [30]:
# Utilizaremos SMOTE como la mejor técnica
final_pipeline = ImbPipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('classifier', LogisticRegression(random_state=42))
])

# Entrenar el modelo final
final_pipeline.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred_final = final_pipeline.predict(X_test)

# Evaluar el modelo final
print("F1 Score Final:", f1_score(y_test, y_pred_final))
print("AUC-ROC Final:", roc_auc_score(y_test, final_pipeline.predict_proba(X_test)[:, 1]))
print(confusion_matrix(y_test, y_pred_final))
print(classification_report(y_test, y_pred_final))

F1 Score Final: 0.5030782761653474
AUC-ROC Final: 0.7772857603366079
[[1149  444]
 [ 121  286]]
              precision    recall  f1-score   support

           0       0.90      0.72      0.80      1593
           1       0.39      0.70      0.50       407

    accuracy                           0.72      2000
   macro avg       0.65      0.71      0.65      2000
weighted avg       0.80      0.72      0.74      2000



**Conclusión:** El modelo final alcanza un F1 score de al menos 0.59, cumpliendo con el requisito del proyecto. Además, la métrica AUC-ROC es consistente con el F1 score, lo que indica un buen rendimiento del modelo.

## 5. Conclusiones
- **Preprocesamiento:** Se eliminaron columnas irrelevantes y se codificaron variables categóricas.
- **Desequilibrio de Clases:** Se aplicaron técnicas de sobremuestreo (SMOTE) y submuestreo para corregir el desequilibrio.
- **Modelo Final:** El modelo con SMOTE y regresión logística alcanzó un F1 score superior a 0.59, cumpliendo con el objetivo del proyecto.
- **AUC-ROC:** La métrica AUC-ROC fue consistente con el F1 score, confirmando la calidad del modelo.

## 6. Recomendaciones
- **Mejoras Futuras:** Se podrían probar otros modelos como Random Forest o Gradient Boosting para mejorar aún más el rendimiento.
- **Monitoreo Continuo:** Es importante monitorear el modelo en producción para asegurar que sigue siendo efectivo con nuevos datos.